<a href="https://colab.research.google.com/github/cekiorpes/big-data-challenge/blob/main/Level_1/Pet_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-19 02:13:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-05-19 02:13:42 (10.5 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PetCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in review data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-01-31 00:08:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [5]:
print(df.count())
df = df.dropna()
print(df.count())

2643619
2643241


In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



Transform data for review_id_table

In [7]:
#Create new DataFrame for review_id_table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886|2015-01-31 00:08:00|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672|2015-01-31 00:08:00|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727|2015-01-31 00:08:00|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110|2015-01-31 00:08:00|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826|2015-01-31 00:08:00|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141|2015-01-31 00:08:00|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603|2015-01-31 00:08:00|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789|2015-01-31 00:08:00|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483|2015-01-31 00:08:00|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658|2015-01-31 00:08:00|
+--------------+---------

In [8]:
#Write DataFrame to RDS
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cojhn4lpsels.us-east-1.rds.amazonaws.com:5432/big_data_db"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [9]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Transform data for products table

In [10]:
#Create new DataFrame for products table
products = df.select(["product_id", "product_title"])
products.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Q0K9604|(8-Pack) EZwhelp ...|
|B00MBW5O9W|Warren Eckstein's...|
|B0084OHUIO|Tyson's True Chew...|
|B001GS71K2|Soft Side Pet Cra...|
|B004ABH1LG|EliteField 3-Door...|
|B00AX0LFM4|Carlson 68-Inch W...|
|B00DQFZGZ0|Dog Seat Cover Wi...|
|B00DIRF9US|The Bird Catcher ...|
|B00JRCBFUG|Cat Bed - Purrfec...|
|B000L3XYZ4|PetSafe Drinkwell...|
+----------+--------------------+
only showing top 10 rows



In [11]:
#Drop duplicates
products = products.dropDuplicates()

In [12]:
#Write DataFrame to RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Transform data for customers table

In [13]:
#Create new DataFrame for customers table
customers = df.select(["customer_id"])
customers.show(5)

+-----------+
|customer_id|
+-----------+
|   28794885|
|   11488901|
|   43214993|
|   12835065|
|   26334022|
+-----------+
only showing top 5 rows



In [14]:
#Count ratings by customer_id
customers = df.groupby("customer_id").agg({"customer_id":"count"})
customers.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   10270641|                 1|
|   18365872|                 1|
|   16711087|                 1|
|   10742726|                 2|
|   41169638|                 1|
+-----------+------------------+
only showing top 5 rows



In [15]:
#Rename count column
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
+-----------+--------------+
only showing top 5 rows



In [27]:
customers = customers.filter((customers.customer_id != 42319435) & (customers.customer_id != 36605391) & (customers.customer_id != 50724762) & (customers.customer_id != 50753128))

In [28]:
#Write DataFrame to RDS
from py4j.protocol import Py4JJavaError

try:
  customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

except Py4JJavaError as err:
  print(str(err))


An error occurred while calling o171.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 28.0 failed 1 times, most recent failure: Lost task 1.0 in stage 28.0 (TID 236, 46684ff6f4e5, executor driver): java.sql.BatchUpdateException: Batch entry 52 INSERT INTO customers ("customer_id","customer_count") VALUES (12160854,3) was aborted: ERROR: duplicate key value violates unique constraint "customers_pkey"
  Detail: Key (customer_id)=(12160854) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:153)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2242)
	at org.postgresql.core.v3.QueryExecutorImpl.flushIfDeadlockRisk(QueryExecutorImpl.java:1357)
	at org.postgresql.core.v3.QueryExecutorImpl.sendQuery(QueryExecutorImpl.java:1382)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:494)
	at org.pos

Transform data for vine_table

In [25]:
#Create new DataFrame for vine_table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [26]:
#Write DataFrame to RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)